# Silver layer

The goal of this notebook is to perform a silver-layer process, where bronze data is read to apply cleaning transformation and the result is stored into a silver repository using delta format.

## 1. Notebook parameters

Parameters, which are called by pipelines.

Inputs:

* **in_parameter_run_id**: run identifier.
* **in_parameter_process_date**: process date.
* **in_parameter_path_storage**: path project storage.
* **in_parameter_path_container**: path project container.
* **in_parameter_bd**: bd name.

Outputs:

* **out_parameter_count_processed**: count of processed rows.

In [1]:
in_parameter_run_id = 0
in_parameter_process_date = "1900-01-01 00:00:00"
in_parameter_path_storage = "datalake20251021"
in_parameter_path_container = "dajobcanada"
in_parameter_bd = "dajobcanada_db"
out_parameter_count_processed = 0

## 2. Load control tables

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

Load **validation_definition** table.

In [3]:
validation_definition_schema = StructType([
    StructField("column_name", StringType(), False),
    StructField("column_type", StringType(), False),
    StructField("column_size", IntegerType(), False),
    StructField("column_scale", IntegerType(), True),
    StructField("default_value", StringType(), False)
    ])

df_validation_definition = spark.read.load(f"abfss://{in_parameter_path_container}@{in_parameter_path_storage}.dfs.core.windows.net/control_db/validation_definition/{in_parameter_run_id}.parquet",
                                            format = "parquet",
                                            schema = validation_definition_schema,
                                            header = False)

Load **job_roles** table.

In [4]:
job_roles_schema = StructType([
    StructField("level", StringType(), False),
    StructField("role", StringType(), False),
    StructField("job_salary_min_base", DecimalType(15, 2), False),
    StructField("job_salary_max_base", DecimalType(15, 2), False)
    ])

df_job_roles = spark.read.load(f"abfss://{in_parameter_path_container}@{in_parameter_path_storage}.dfs.core.windows.net/control_db/job_roles/{in_parameter_run_id}.parquet",
                                format = "parquet",
                                schema = job_roles_schema,
                                header = False)

## 3. Read bronze data

Filter data by date of process (partition filtering)

In [5]:
df = spark.read.load(f"abfss://{in_parameter_path_container}@{in_parameter_path_storage}.dfs.core.windows.net/bronze/jobs/process_date={in_parameter_process_date[:10]}/*.parquet",
                    format = "parquet",
                    header = True)

Filter data by id of process (in case there are several runs in one day)

In [6]:
df = df.filter(col("run_id") == in_parameter_run_id)

## 4. Cleaning and transformation

In [7]:
log_schema = StructType([
    StructField("id_run", ShortType(), False),
    StructField("description", StringType(), False)
])

### 4.1 Uppercase

To standarize text and avoid any mismatch in join and filtering. I'm skipping semantic meaning for this project. The exception is "link job", where it is important to preserve the original text (semantic meaning).

In [8]:
df = df.withColumn("job_title", upper(df["job_title"]))
df = df.withColumn("company_name", upper(df["company_name"]))
df = df.withColumn("language_tools", upper(df["language_tools"]))
df = df.withColumn("city", upper(df["city"]))
df = df.withColumn("province", upper(df["province"]))
df = df.withColumn("job_salary", upper(df["job_salary"]))

### 4.2. Trimming

To standarize text and avoid any mismatch in join and filtering.

In [9]:
df = df.withColumn("job_title", trim(df["job_title"]))
df = df.withColumn("company_name", trim(df["company_name"]))
df = df.withColumn("language_tools", trim(df["language_tools"]))
df = df.withColumn("job_salary", trim(df["job_salary"]))
df = df.withColumn("city", trim(df["city"]))
df = df.withColumn("province", trim(df["province"]))
df = df.withColumn("job_salary", trim(df["job_salary"]))

### 4.3. Fill missing values

The word "UNKNOWN" is the standard value for missing values.

#### 4.3.1. Identify missing values

In [10]:
job_title_count_missing = df.filter(col("job_title").isNull()).count()
company_name_count_missing = df.filter(col("company_name").isNull()).count()
language_tools_count_missing = df.filter(col("language_tools").isNull()).count()
city_count_missing = df.filter(col("city").isNull()).count()
province_count_missing = df.filter(col("province").isNull()).count()
job_salary_count_missing = df.filter(col("job_salary").isNull()).count()
job_link_count_missing = df.filter(col("job_link").isNull()).count()

In [11]:
data = [(in_parameter_run_id, "job_title missing values count: " + str(job_title_count_missing)), 
        (in_parameter_run_id, "company_name missing values count: " + str(company_name_count_missing)),
        (in_parameter_run_id, "language_tools missing values count: " + str(language_tools_count_missing)),
        (in_parameter_run_id, "city missing values count: " + str(city_count_missing)),
        (in_parameter_run_id, "province missing values count: " + str(province_count_missing)),
        (in_parameter_run_id, "job_salary missing values count: " + str(job_salary_count_missing)),
        (in_parameter_run_id, "job_link missing values count: " + str(job_link_count_missing)),
        ]
df_logs_missing_values = spark.createDataFrame(data, log_schema)

#### 4.3.2. Fill missing values

In [12]:
job_title_default_v = df_validation_definition.filter(col("column_name") == "job_title").select("default_value").first()[0]
company_name_default_v = df_validation_definition.filter(col("column_name") == "company_name").select("default_value").first()[0]
language_tools_default_v = df_validation_definition.filter(col("column_name") == "language_tools").select("default_value").first()[0]
city_default_v = df_validation_definition.filter(col("column_name") == "city").select("default_value").first()[0]
province_default_v = df_validation_definition.filter(col("column_name") == "province").select("default_value").first()[0]
job_salary_default_v = df_validation_definition.filter(col("column_name") == "job_salary").select("default_value").first()[0]
job_link_default_v = df_validation_definition.filter(col("column_name") == "job_link").select("default_value").first()[0]

In [13]:
df = df.fillna({"job_title": job_title_default_v})
df = df.fillna({"company_name": company_name_default_v})
df = df.fillna({"language_tools": language_tools_default_v})
df = df.fillna({"city": city_default_v})
df = df.fillna({"province": province_default_v})
df = df.fillna({"job_salary": job_salary_default_v})
df = df.fillna({"job_link": job_link_default_v})

### 4.4. Remove special Characters

To standarize text and avoid any mismatch in join and filtering. The exception is "link job", where it is important to preserve the original text (semantic meaning).

In [14]:
from pyspark.sql.functions import regexp_replace

pattern = "[^a-zA-Z0-9\\s]" # only letters and numbers
df = df.withColumn("job_title", regexp_replace(col("job_title"), pattern, ""))
df = df.withColumn("company_name", regexp_replace(col("company_name"), pattern, ""))
df = df.withColumn("language_tools", regexp_replace(col("language_tools"), pattern, ""))
df = df.withColumn("city", regexp_replace(col("city"), pattern, ""))
df = df.withColumn("province", regexp_replace(col("province"), pattern, ""))

### 4.5. Extract numbers

The job salary has text and numbers, but it's the numeric value is necessary.

Remove "," to facilitate number extraction.

In [15]:
df = df.withColumn("job_salary", regexp_replace("job_salary", ",", ""))

Apply regular expressions to extract numbers, where two groups were identified (min and max), due to job salary was define like a range.

In [16]:
from pyspark.sql.functions import regexp_extract

# Define a regular expression to capture two numbers
# (\d+\.?\d*) captures one or more digits, optionally followed by a decimal point and more digits
# The first group (index 1) will be the first number, the second group (index 2) the second number
pattern = r".*?(\d+\.?\d*).*?(\d+\.?\d*).*"

df = df.withColumn("job_salary_min", regexp_extract(col("job_salary"), pattern, 1))
df = df.withColumn("job_salary_max", regexp_extract(col("job_salary"), pattern, 2))

If there is no value, "0" takes place.

In [17]:
job_salary_min_default_v = df_validation_definition.filter(col("column_name") == "job_salary_min").select("default_value").first()[0]
job_salary_max_default_v = df_validation_definition.filter(col("column_name") == "job_salary_max").select("default_value").first()[0]

In [18]:
df = df.withColumn("job_salary_min", 
                when(col("job_salary_min") == "", lit(job_salary_min_default_v)).
                otherwise(col("job_salary_min"))
                )

df = df.withColumn("job_salary_max", 
                when(col("job_salary_max") == "", lit(job_salary_max_default_v)).
                otherwise(col("job_salary_max"))
                )

### 4.6. Create type payment

It was identified that several types of salaries take place (hourly, yearly, monthly, etc), but it's embedded in "job_salary" column, so a new column is created to make easier its identification.

In [19]:
job_salary_type_default_v = df_validation_definition.filter(col("column_name") == "job_salary_type").select("default_value").first()[0]

In [20]:
df = df.withColumn("job_salary_type", 
                when(col("job_salary").like("%HOUR%") == True, lit("H")).
                when(col("job_salary").like("%YEAR%") == True, lit("Y")).
                when(col("job_salary").like("%MONTH%") == True, lit("M")).
                otherwise(lit(job_salary_type_default_v))
                )

### 4.7. Standarize remote position

There are some jobs where can be done remotely, but some jobs provide information that is not necessary just adding noise, so they are removed.

Fill with "REMOTE" all jobs can be done by that way.

In [21]:
df = df.withColumn("city", when(col("city").like("%REMOTE%") == True, lit("REMOTE")).otherwise(col("city")))

In case of provinces, where jobs can be done remotely, a new category was created "EVERYWHERE".

In [22]:
df = df.withColumn("province", when(col("city").like("%REMOTE%") == True, lit("EVERYWHERE")).otherwise(col("city")))

### 4.8. Create experience level

A new feature is created to identify the experience required by one specific job position.

In [23]:
level_default_v = df_validation_definition.filter(col("column_name") == "level").select("default_value").first()[0]

In [24]:
df = df.withColumn("level", 
                    when((col("job_title").like("%SR%") == True) | (col("job_title").like("%SENIOR%") == True), lit("SENIOR")).
                    when(col("job_title").like("%LEAD%") == True, lit("LEAD")).
                    when(col("job_title").like("%SPECIALIST%") == True, lit("SPECIALIST")).
                    otherwise(lit(level_default_v)))

### 4.9. Create role

A new feature is created to identify the role required by one specific job position.

In [25]:
role_default_v = df_validation_definition.filter(col("column_name") == "role").select("default_value").first()[0]

In [26]:
df = df.withColumn("role", 
                    when(col("job_title").like("%ANALYST%") == True, lit("ANALYST")).
                    when(col("job_title").like("%DEVELOPER%") == True, lit("DEVELOPER")).
                    when(col("job_title").like("%ENGINEER%") == True, lit("ENGINEER")).
                    otherwise(lit(role_default_v)))

## 5. Data integrity

This step focus on data type validation.

### 5.1. Numeric values

Apply cast to decimal and null evaluation to job salaries in order to know which pass validation.

In [27]:
job_salary_min_size = df_validation_definition.filter(col("column_name") == "job_salary_min").select("column_size", "column_scale").first()
job_salary_max_size = df_validation_definition.filter(col("column_name") == "job_salary_max").select("column_size", "column_scale").first()

In [28]:
df = df.withColumn("job_salary_min_is_number", col("job_salary_min").cast(DecimalType(job_salary_min_size[0], job_salary_min_size[1])).isNotNull())
df = df.withColumn("job_salary_max_is_number", col("job_salary_max").cast(DecimalType(job_salary_max_size[0], job_salary_max_size[1])).isNotNull())

Filter cases that do not pass validation (log).

In [29]:
job_salary_min_no_number = df.filter(col("job_salary_min_is_number") == False).count()
job_salary_max_no_number = df.filter(col("job_salary_max_is_number") == False).count()

Filter only cases that pass validation.

In [30]:
df = df.filter((col("job_salary_min_is_number") == True) & (col("job_salary_max_is_number") == True))

Cast data types

In [31]:
df = df.withColumn("job_salary_min", col("job_salary_min").cast(DecimalType(job_salary_min_size[0], job_salary_min_size[1])))
df = df.withColumn("job_salary_max", col("job_salary_max").cast(DecimalType(job_salary_max_size[0], job_salary_max_size[1])))

### 5.2. String values

#### 5.2.1. Identify length no valid

To validate column sizes.

In [32]:
job_title_size = df_validation_definition.filter(col("column_name") == "job_title").select("column_size").first()[0]
company_name_size = df_validation_definition.filter(col("column_name") == "company_name").select("column_size").first()[0]
language_tools_size = df_validation_definition.filter(col("column_name") == "language_tools").select("column_size").first()[0]
job_salary_size = df_validation_definition.filter(col("column_name") == "job_salary").select("column_size").first()[0]
city_size = df_validation_definition.filter(col("column_name") == "city").select("column_size").first()[0]
province_size = df_validation_definition.filter(col("column_name") == "province").select("column_size").first()[0]
job_link_size = df_validation_definition.filter(col("column_name") == "job_link").select("column_size").first()[0]
level_size = df_validation_definition.filter(col("column_name") == "level").select("column_size").first()[0]
role_size = df_validation_definition.filter(col("column_name") == "role").select("column_size").first()[0]

In [33]:
df = df.withColumn("job_title_length", length(col("job_title")))
df = df.withColumn("company_name_length", length(col("company_name")))
df = df.withColumn("language_tools_length", length(col("language_tools")))
df = df.withColumn("city_length", length(col("city")))
df = df.withColumn("province_length", length(col("province")))
df = df.withColumn("job_link_length", length(col("job_link")))
df = df.withColumn("level_length", length(col("level")))
df = df.withColumn("role_length", length(col("role")))

In [34]:
job_title_length_no_valid = df.filter(col("job_title_length") > job_title_size).count()
company_name_length_no_valid = df.filter(col("company_name_length") > company_name_size).count()
language_tools_length_no_valid = df.filter(col("language_tools_length") > language_tools_size).count()
city_length_no_valid = df.filter(col("city_length") > city_size).count()
province_length_no_valid = df.filter(col("province_length") > province_size).count()
job_link_length_no_valid = df.filter(col("job_link_length") > job_link_size).count()
level_length_no_valid = df.filter(col("level_length") > level_size).count()
role_length_no_valid = df.filter(col("role_length") > role_size).count()

In [35]:
data = [(in_parameter_run_id, "job_salary_min no number count: " + str(job_salary_min_no_number)), 
        (in_parameter_run_id, "job_salary_max no number count: " + str(job_salary_max_no_number)),
        (in_parameter_run_id, "job_title length no valid count: " + str(job_title_length_no_valid)),
        (in_parameter_run_id, "city_length length no valid count: " + str(city_length_no_valid)),
        (in_parameter_run_id, "province length no valid count: " + str(province_length_no_valid)),
        (in_parameter_run_id, "job_link length no valid count: " + str(job_link_length_no_valid)),
        (in_parameter_run_id, "level length no valid count: " + str(level_length_no_valid)),
        (in_parameter_run_id, "role length no valid count: " + str(role_length_no_valid))
        ]
df_logs_data_integrity = spark.createDataFrame(data, log_schema)

#### 5.2.2. Select length valid

In [36]:
df = df.filter((col("job_title_length") <= job_title_size) & \
                (col("company_name_length") <= company_name_size) & \
                (col("language_tools_length") <= language_tools_size) & \
                (col("city_length") <= city_size) & \
                (col("province_length") <= province_size) & \
                (col("job_link_length") <= job_link_size) & \
                (col("level_length") <= level_size) & \
                (col("role_length") <= role_size)
)

## 6. Fill job salary

In [37]:
df = df.join(df_job_roles, on = ["level", "role"], how = "inner")

In [38]:
df = df.withColumn("job_salary_min", 
                    when(col("job_salary_min") == 0, col("job_salary_min_base")).
                    otherwise(col("job_salary_min")))

df = df.withColumn("job_salary_max", 
                    when(col("job_salary_max") == 0, col("job_salary_max_base")).
                    otherwise(col("job_salary_max")))

## 7. Save log

In [39]:
df_logs = df_logs_missing_values.unionAll(df_logs_data_integrity)

In [41]:
df_logs.write.mode("overwrite") \
            .format("parquet") \
            .save(f"abfss://{in_parameter_path_container}@{in_parameter_path_storage}.dfs.core.windows.net/control_db/log_validation/{in_parameter_run_id}/")

## 8. Save results

Select final columns.

In [42]:
df = df.select("job_id", "job_title", "company_name", "language_tools", "city", "province", "job_link",
                "level",  "role", "job_salary_min", "job_salary_max", "job_salary_type", "run_id", "process_datetime")

Save silver table.

In [43]:
df = df.withColumn("process_date", lit(in_parameter_process_date[:10]))

In [44]:
df.write.partitionBy("process_date") \
                .mode("append") \
                .format("delta") \
                .save(f"abfss://{in_parameter_path_container}@{in_parameter_path_storage}.dfs.core.windows.net/silver/jobs/")

Identify the number of processed rows.

In [45]:
out_parameter_count_processed = df.count()

In [46]:
mssparkutils.notebook.exit(out_parameter_count_processed)